In [ ]:
!pip install xlsxwriter
import pandas as pd
import numpy as np
from google.colab import files
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statistics
import xlsxwriter
import random
import datetime
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
file_name=files.upload()
name='Economic.csv'

Saving Economic.csv to Economic.csv


In [ ]:
## Checking that file uploaded correctly 
print(name)

['Economic.csv']


# Processing the data





In [66]:
## Creating a variable for the name of output file so several different versions can be created after modificactions 
output_file_name="LEI_CEI_analysis_V1"
output_file_description="-" #optional file description 

In [67]:
## Reading the file in 
df=pd.read_csv(name, sep=';')
rows=df.shape[0] #number of rows
columns=df.shape[1]  #number of columns
print(rows)
print (columns)
df.head()

600
7


,month,LEI,CEI,GDP,NBER,Fed_Funds,Inflation
0,7/01/1972,50.3,39.5,3.8,expansion,4.55,2.9
1,8/01/1972,51.0,39.9,3.8,expansion,4.81,2.9
2,9/01/1972,51.6,40.2,3.8,expansion,4.87,3.2
3,10/01/1972,52.0,40.7,6.9,expansion,5.05,3.4
4,11/01/1972,52.5,41.0,6.9,expansion,5.06,3.7


**Comment on datatypes and what the variables mean

In [68]:
## checking if the datatypes correspond
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   month      600 non-null    object 
 1   LEI        600 non-null    float64
 2   CEI        600 non-null    float64
 3   GDP        600 non-null    float64
 4   NBER       600 non-null    object 
 5   Fed_Funds  600 non-null    float64
 6   Inflation  600 non-null    float64
dtypes: float64(5), object(2)
memory usage: 32.9+ KB


In [69]:
## Changing month to datetime format. 
df['month']=df['month'].astype('datetime64')

In [70]:
df['month'] = df['month'].dt.strftime('%m/%Y')

In [71]:
df.head()

,month,LEI,CEI,GDP,NBER,Fed_Funds,Inflation
0,07/1972,50.3,39.5,3.8,expansion,4.55,2.9
1,08/1972,51.0,39.9,3.8,expansion,4.81,2.9
2,09/1972,51.6,40.2,3.8,expansion,4.87,3.2
3,10/1972,52.0,40.7,6.9,expansion,5.05,3.4
4,11/1972,52.5,41.0,6.9,expansion,5.06,3.7


In [72]:
## These are the change variables we want to include in the dataframe for analysis
delta_1m_LEI=0   ## change in one month
delta_3m_LEI=0   ## change in three months
delta_6m_LEI=0   ## change in six months
delta_1y_LEI=0   ## change in a year

delta_1m_CEI=0   ## change in one month
delta_3m_CEI=0   ## change in three months
delta_6m_CEI=0   ## change in six months
delta_1y_CEI=0   ## change in a year

## Lists to store the values of estimations to later include in the dataframe
list_month=[df["month"][0]] # we exclude the first row because no previous data exists
list_LEI=[df["LEI"][0]]
list_CEI=[df["CEI"][0]]
list_NBER=[df["NBER"][0]]
list_delta_1m_LEI=[0]
list_delta_3m_LEI=[0]
list_delta_6m_LEI=[0]
list_delta_1y_LEI=[0]
list_delta_1m_CEI=[0]
list_delta_3m_CEI=[0]
list_delta_6m_CEI=[0]
list_delta_1y_CEI=[0]





In [73]:
### Estimating changes in LEI and appending to list 
for i in range(1,rows,1):
  delta_1m_LEI=((df["LEI"][i]-df["LEI"][i-1])/df["LEI"][i-1])
  if i>=3:
    delta_3m_LEI=((df["LEI"][i]-df["LEI"][i-3])/df["LEI"][i-3])
  if i>=6:
    delta_6m_LEI=((df["LEI"][i]-df["LEI"][i-6])/df["LEI"][i-6])
  if i>=12:
    delta_1y_LEI=((df["LEI"][i]-df["LEI"][i-12])/df["LEI"][i-12])

  list_delta_1m_LEI.append(delta_1m_LEI)
  list_delta_3m_LEI.append(delta_3m_LEI)
  list_delta_6m_LEI.append(delta_6m_LEI)
  list_delta_1y_LEI.append(delta_1y_LEI)

## Estimating changes to CEI and appending to list 
  delta_1m_CEI=((df["CEI"][i]-df["CEI"][i-1])/df["CEI"][i-1])
  if i>=3:
    delta_3m_CEI=((df["CEI"][i]-df["CEI"][i-3])/df["CEI"][i-3])
  if i>=6:
    delta_6m_CEI=((df["CEI"][i]-df["CEI"][i-6])/df["CEI"][i-6])
  if i>=12:
    delta_1y_CEI=((df["CEI"][i]-df["CEI"][i-12])/df["CEI"][i-12])

  list_delta_1m_CEI.append(delta_1m_CEI)
  list_delta_3m_CEI.append(delta_3m_CEI)
  list_delta_6m_CEI.append(delta_6m_CEI)
  list_delta_1y_CEI.append(delta_1y_CEI)  
  
## extracting the values for new csv created with the deltas   
  list_month.append(df["month"][i]) 
  list_LEI.append(df["LEI"][i])
  list_CEI.append(df["CEI"][i]) 
  list_NBER.append(df["NBER"][i]) 

In [74]:
### creating new csv file with calculations

direc="/content/"
text=output_file_name+'.csv'
data_output={
  
    "month":list_month,
    "LEI": list_LEI,
    "CEI":list_CEI,
    "NBER": list_NBER,
    "delta_1m_LEI":list_delta_1m_LEI,
    "delta_3m_LEI":list_delta_3m_LEI,
    "delta_6m_LEI":list_delta_6m_LEI,
    "delta_1y_LEI":list_delta_1y_LEI,
    "delta_1m_CEI":list_delta_1m_CEI,
    "delta_3m_CEI":list_delta_3m_CEI,
    "delta_6m_CEI":list_delta_6m_CEI,
    "delta_1y_CEI":list_delta_1y_CEI,
 
    }
 
df2 = pd.DataFrame(data=data_output)
df2.to_csv(text,index=False) 
file_name=direc+text
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>